# Parabolic Focus Determination Data Acquisition

This notebook executes a parabolic focus data acquisition for a specific filter/grating configuration. It performs fine sampling of z-axis hexapod translations and takes images at each hexapod position. The analysis will be done in a separate notebook. 

Also contained in this notebook are three focus script executions for the configurations filter/grating, filter/no grating and no filter/no grating. In some cases they might be useful as a comparison with the results of the parabolic focus results. 

# Notebook dependency set up

## Import libraries

In [ ]:
import sys
import asyncio
import time
import os
import datetime 

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_wep_align import LatissWEPAlign
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.idl.enums.Script import ScriptState

## Setting up logger

In [ ]:
logger = logging.getLogger("Parabolic_Focus")
logger.level = logging.DEBUG

## Getting unique index for script

Restart from here if you want to execute the parabolic focus sequence again. This will generate a new script index. 

In [ ]:
index = int((datetime.datetime.today().strftime('%Y%m%d')) + str(np.random.randint(10, 99)))
logger.info(f'The generated index is {index}')

## Instantiate Alignment and Focus Script

In many cases, it is desirable to perform a focus sequence for comparison with the parabolic focus. <br>
This is because the focus offsets are measured relative to the empty position in each wheel. <br>
Therefore, LatissWEPAlign script is instantiated, to provide the option to the user after the parabolic focus data acquisition is done. <br>
However, it can also be run beforehand if desired.

In [ ]:
script = LatissWEPAlign(index=index, remotes=True)  # this essentially calls the init method
await script.start_task

## Forward ATCS and LATISS

In [ ]:
atcs = script.atcs
latiss = script.latiss

## Set up script log level

In [ ]:
script.log.level = logging.DEBUG

---
# Parabolic Focus Data Acquisition

## Configuration

### ATSpectrograph configuration

Get available instrument configurations:

In [ ]:
inst_setup = await script.latiss.get_available_instrument_setup()
logger.info(f'Filters are: {inst_setup[0]},\nGratings are: {inst_setup[1]}')

Declare the configuration of the instrument for the focus determination sequence. Both variables are strings. 

In [ ]:
filter_to_focus = 'SDSSr_65mm'
grating_to_focus = 'holo4_003'

### ATAOS corrections

ATAOS corrections must be enabled for this test. In the cell below, ATAOS corrections will be enabled, in case they were not. 

In [ ]:
corrections_enabled = await script.atcs.rem.ataos.evt_correctionEnabled.aget()
if not (corrections_enabled.m1 and corrections_enabled.hexapod and corrections_enabled.atspectrograph):
    cmd = await script.atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)
    logger.info(f'ATAOS corrections enabled: {cmd.result}')
else:
    logger.info(f'ATAOS corrections already enabled')

## Slew to target

### Declare target 

Declare target name

In [ ]:
#target = 'HD '

or query for a target at a user-defined azimuth and elevation

In [ ]:
target = await script.atcs.find_target(az=90, el=60, mag_limit=8)
logger.info(f'The target is {target}')

### Slew to the defined target

In [ ]:
await script.atcs.slew_object(target, rot_type=RotType.Parallactic)

Take test image (also to ensure the right filter and grating are on) 

In [ ]:
await script.latiss.take_engtest(exptime=2, filter=filter_to_focus, grating=grating_to_focus, program="Parabolic_Focus")

### Acquire target

#### With grating in the beam

If a grating is used, you need to acquire the target with these additional steps. 
Note that a new grating will require that the desired star placement has been set via the ATCS.

As an interim solution, run `latiss_acquire_and_take_sequence` script from the ATQueue with the following configuration. Change the `object_name` with the target you are currently tracking, `acq_filter` with the filter and `acq_grating` with the grating in use.  

            acq_exposure_time: 2.0
            acq_filter: ***** CHANGE ME *****
            acq_grating: ***** CHANGE ME *****
            do_acquire: true
            do_pointing_model: false
            max_acq_iter: 3
            object_name: ***** CHANGE ME *****
            target_pointing_tolerance: 5.0
            target_pointing_verification: true


#### Without grating

Uncomment the following line to take a snapshot to verify the target is positioned as expected 

In [ ]:
#await script.latiss.take_engtest(exptime=5, filter=filter_to_focus, grating=grating_to_focus)

To offset the telescope and center the source (if required) uncomment the following line. <br>
Offsets are in detector X/Y coordinates and in arcseconds. 

In [ ]:
# await script.atcs.offset_xy(x=20, y=20)

### Acquiring Data

Acquire data in steps of `z_offset_step` in the range of `z_offset_start` to `z_offset_end` relative to the current focus position.

In [ ]:
z_offset_start = -0.1 # mm
z_offset_step = 0.025 # mm
z_offset_end = -z_offset_start

steps = np.arange(z_offset_start, z_offset_end + 0.01, z_offset_step)

logger.info(f'The hexapod will be moved the following z offset steps \n {steps} [mm]. \n If you disagree, change the parameters above and run this cell again')

In [ ]:
original_focus_offset = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
script.log.info(f'Original focus offset is {original_focus_offset}')

In [ ]:
script.log.info(f'START -- Parabolic_Focus -- {filter_to_focus} and {grating_to_focus} '
                    f'Time UTC -- {astropy.time.Time(astropy.time.Time.now())}') 

In [ ]:
parabolic_focus_data = []

await atcs.rem.ataos.cmd_offset.set_start(z=z_offset_start)
    
await asyncio.sleep(2)
for step in steps:
        
    exposure = await latiss.take_engtest(
        exptime=20, n=1, filter=filter_to_focus, grating=grating_to_focus, reason='focus_sequence', program="Parabolic_Focus")
    parabolic_focus_data.append(exposure[0])
    script.log.info(f'Total z offset = {step:.3f} mm -- Image expId = {exposure[0]}')
    
    logger.info(f'Current focus offset is \n {await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()}')
    
    await atcs.rem.ataos.cmd_offset.set_start(z=z_offset_step)

# Move back to the original focus offset position
await atcs.rem.ataos.cmd_offset.set_start(z= -(z_offset_end + z_offset_step))
logger.info(f'Back to the original focus offset position \n {await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()}')

# Log results into EFD for later analysis
script.log.info(f'END -- Parabolic_Focus -- {filter_to_focus} and {grating_to_focus}  '
                    f'Images expId -- {parabolic_focus_data} '
                    f'Time UTC -- {astropy.time.Time(astropy.time.Time.now())}') 

In [ ]:
current_focus_offset = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
script.log.info(f'Current focus offset is {current_focus_offset}')

Original and current focus offset's user applied should be now equal.  

In [ ]:
diff = current_focus_offset.userApplied - original_focus_offset.userApplied
print(f'Difference between current and original user Applied focus is {diff:0.2f} mm')

Save this execution notebook for later use during the data analysis, as the `test_date`, `index` and `original_focus_offset` will be needed in the parabolic_Focus_Analysis.ipynb notebook.

## Stop tracking

If required, then uncomment and use the following cell to stop the telescope from tracking, but you will lose your acquisition.

In [ ]:
await script.atcs.stop_tracking()

# Mirror Alignment and Focus
Below are the focusing executions for the combinations filter & grating, filter & no grating and no filter & no grating. 

## Mirror Alignment and focus with a filter and grating

### Set up configuration

In [ ]:
configuration = yaml.safe_dump({"filter": filter_to_focus, 
                                "grating": grating_to_focus,
                                "exposure_time": 20,})

The next line is not required the first time the script is run, however, in each additional instance the cell is run, an error will be thrown if it is not included.  
Therefore, it is included here despite being a non-operation in the first instance.  

In [ ]:
await script.set_state(ScriptState.UNCONFIGURED)

### Put the ScriptState to CONFIGURED

In [ ]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

Set these script parameters to None to verify a re-reduction does not happen of the images.

In [ ]:
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10

### Set groupID and launch the script

This sets the same group ID for all exposures taken in the script.

In [ ]:
group_id_data = script.cmd_setGroupId.DataType(
    groupId=astropy.time.Time.now().isot)

await script.do_setGroupId(group_id_data)
await script.arun()

In [ ]:
current_focus_offset_with_filter_with_grating = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
script.log.info(f'Current focus with filter/with grating {current_focus_offset_with_filter_with_grating}')

## Mirror Alignment and focus with a filter but no grating

### Set up configuration

In [ ]:
configuration = yaml.safe_dump({"filter": filter_to_focus, 
                                "grating": 'empty_1',
                                "exposure_time": 20,})

The next line is not required the first time the script is run, however, in each additional instance the cell is run, an error will be thrown if it is not included.  
Therefore, it is included here despite being a non-operation in the first instance.  

In [ ]:
await script.set_state(ScriptState.UNCONFIGURED)

### Put the ScriptState to CONFIGURED

In [ ]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

Set these script parameters to None to verify a re-reduction does not happen of the images.

In [ ]:
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10

### Set groupID and launch the script

This sets the same group ID for all exposures taken in the script.

In [ ]:
group_id_data = script.cmd_setGroupId.DataType(
    groupId=astropy.time.Time.now().isot)

await script.do_setGroupId(group_id_data)
await script.arun()

In [ ]:
current_focus_offset_without_grating = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
script.log.info(f'Current focus with filter/without grating {current_focus_offset_without_grating}')

## Mirror Alignment and focus without a grating and filter

### Set up configuration

In [ ]:
configuration = yaml.safe_dump({"filter": 'empty_1', 
                                "grating": 'empty_1',
                                "exposure_time": 20,})

The next line is not required the first time the script is run, however, in each additional instance the cell is run, an error will be thrown if it is not included.  
Therefore, it is included here despite being a non-operation in the first instance.  

In [ ]:
await script.set_state(ScriptState.UNCONFIGURED)

### Put the ScriptState to CONFIGURED

In [ ]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

Set these script parameters to None to verify a re-reduction does not happen of the images.

In [ ]:
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10

### Set groupID and launch the script

This sets the same group ID for all exposures taken in the script.

In [ ]:
group_id_data = script.cmd_setGroupId.DataType(
    groupId=astropy.time.Time.now().isot)

await script.do_setGroupId(group_id_data)
await script.arun()

In [ ]:
current_focus_offset_without_filter_without_grating = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
script.log.info(f'Current focus without filter/without grating {current_focus_offset_without_filter_without_grating}')

## Stop tracking

If required, then uncomment and use the following cell to stop the telescope from tracking, but you will lose your acquisition.

In [ ]:
await script.atcs.stop_tracking()